In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [12]:
RESPONSE_JSON = {
    "1": {
        "question":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct_answer"
    },
    "2": {
        "question":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct_answer"
    },
    "3": {
        "question":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct_answer"
    },
}

In [8]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} subject with {tone} tone.
Make sure that the questions should not be repeated and check all the questions conforming the text as well.
Make sure you format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","subject","number","tone","response_json"],
    template=TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

g:\LLM-MCQ-Generator\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
EVALUATION_TEMPLATE = """
You are an expert grammarian and writer. Given a multiple choice quiz for {subject} subject.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the analysis. 
If the quiz is not as per the cognitive and thinkig ability of the student then update the quiz questions that need to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}
"""

In [15]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=EVALUATION_TEMPLATE)

In [16]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [17]:
generate_quiz_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","subject","number","tone","response_json"],output_variables=["quiz","review"],verbose=True)

In [18]:
file_path="G:\LLM-MCQ-Generator\data.txt"

with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

Machine Learning is defined as a technology that is used to train machines to perform various actions such as predictions, recommendations, estimations, etc., based on historical data or past experience.

Machine Learning enables computers to behave like human beings by training them with the help of past experience and predicted data.

There are three key aspects of Machine Learning, which are as follows:

Task: A task is defined as the main problem in which we are interested. This task/problem can be related to the predictions and recommendations and estimations, etc.
Experience: It is defined as learning from historical or past data and used to estimate and resolve future tasks.
Performance: It is defined as the capacity of any machine to resolve any machine learning task or problem and provide the best outcome for the same. However, performance is dependent on the type of machine learning problems.


In [19]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "2": {"question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "3": {"question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}}'

In [45]:
SUBJECT = "MACHINE LEARNING",
NUMBER = 2,
TONE = "EASY"

In [46]:
# Setting up the token 
with get_openai_callback() as cb:
    response = generate_quiz_chain(
        {
        "text": TEXT,
        "subject": SUBJECT,
        "number": NUMBER,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine Learning is defined as a technology that is used to train machines to perform various actions such as predictions, recommendations, estimations, etc., based on historical data or past experience.

Machine Learning enables computers to behave like human beings by training them with the help of past experience and predicted data.

There are three key aspects of Machine Learning, which are as follows:

Task: A task is defined as the main problem in which we are interested. This task/problem can be related to the predictions and recommendations and estimations, etc.
Experience: It is defined as learning from historical or past data and used to estimate and resolve future tasks.
Performance: It is defined as the capacity of any machine to resolve any machine learning task or problem and provide the best outcome for the same. However, performance is dependent on the type of mac

In [47]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 882
Prompt Tokens: 663
Completion Tokens: 219
Total Cost: 0.0014325


In [55]:
quiz = response['quiz']

In [56]:
quiz = json.loads(quiz)
print(quiz)

{'1': {'question': 'What is the main problem in which we are interested in Machine Learning?', 'options': {'a': 'Experience', 'b': 'Task', 'c': 'Performance', 'd': 'Prediction'}, 'correct': 'b'}, '2': {'question': 'What is defined as learning from historical or past data in Machine Learning?', 'options': {'a': 'Task', 'b': 'Performance', 'c': 'Experience', 'd': 'Estimation'}, 'correct': 'c'}}


In [57]:
quiz_table_data = []
for key,value in quiz.items():
    question = value["question"]
    options = " | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"question":question,"options":options,"correct":correct})

In [58]:
quiz_table_data

[{'question': 'What is the main problem in which we are interested in Machine Learning?',
  'options': 'a:Experience | b:Task | c:Performance | d:Prediction',
  'correct': 'b'},
 {'question': 'What is defined as learning from historical or past data in Machine Learning?',
  'options': 'a:Task | b:Performance | c:Experience | d:Estimation',
  'correct': 'c'}]

In [62]:
quiz=pd.DataFrame(quiz_table_data)
print(quiz)

                                            question  \
0  What is the main problem in which we are inter...   
1  What is defined as learning from historical or...   

                                             options correct  
0  a:Experience | b:Task | c:Performance | d:Pred...       b  
1  a:Task | b:Performance | c:Experience | d:Esti...       c  


In [60]:
quiz.to_csv('machinelearningquiz.csv')